<a href="https://colab.research.google.com/github/Georgeman31/std_projects/blob/main/NLP/TokenClassification_ProtasovEA_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip /content/FactRuEval.zip

Archive:  /content/FactRuEval.zip
  inflating: dev.txt                 
  inflating: test.txt                
  inflating: train.txt               


# Импорт библиотек

In [2]:
!pip install transformers datasets seqeval evaluate huggingface_hub accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py

In [54]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import re
import csv

from datasets import Dataset
from datasets.features import ClassLabel

from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM, 
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    get_scheduler,
    pipeline
)

import torch
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from torch.optim import AdamW

from accelerate import Accelerator

import evaluate

from huggingface_hub import notebook_login
from huggingface_hub import Repository, get_full_repo_name

In [4]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


# Считывание файлов с данными

In [6]:
with open('/content/train.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('train.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('title', 'intro'))
        writer.writerows(lines)
with open('/content/dev.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('val.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('title', 'intro'))
        writer.writerows(lines)
with open('/content/test.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('test.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('title', 'intro'))
        writer.writerows(lines)

# Предобработка данных

In [7]:
DATA_TRAIN = pd.read_csv('/content/train.csv', sep='\n')
DATA_VAL = pd.read_csv('/content/val.csv', sep='\n')

TOKEN_LIST = []
NER_TAG_LIST = []

tmp1_list = []
tmp2_list = []
list_with_seq = []
list_with_ner = []
count_place = []
count = 0

train_df,val_df  = pd.DataFrame(), pd.DataFrame()

In [8]:
def element_distributor(element):
    element_after_distributor = element.split(' ')
    TOKEN_LIST.append(element_after_distributor[0])
    NER_TAG_LIST.append(element_after_distributor[1])

def function(DATA, df):
  DATA[DATA.columns[0]].apply(element_distributor)
  for token,tag in zip(TOKEN_LIST, NER_TAG_LIST):
      tmp1_list=[]
      tmp2_list=[]
      tmp1_list.append(token)
      tmp2_list.append(tag)
      if token == '.':
          list_with_seq.append(tmp1_list)
          list_with_ner.append(tmp2_list)
  df['ner_tags'] = list_with_ner
  df['tokens'] = list_with_seq
  return df 

In [10]:
DATA_TRAIN['title,intro'].apply(element_distributor)
print('ready')

ready


In [11]:
for token,tag in zip(TOKEN_LIST, NER_TAG_LIST):
  tmp1_list.append(token)
  tmp2_list.append(tag)
  if token == '.':
    list_with_seq.append(tmp1_list)
    list_with_ner.append(tmp2_list)
    tmp1_list=[]
    tmp2_list=[]
    

In [12]:
train_df['ner_tags'] = list_with_ner
train_df['tokens'] = list_with_seq

In [13]:
TOKEN_LIST = []
NER_TAG_LIST = []

tmp1_list = []
tmp2_list = []
list_with_seq = []
list_with_ner = []

In [14]:
DATA_TEST = pd.read_csv('/content/test.csv', sep='\n')
test_df = pd.DataFrame()

In [15]:
TOKEN_LIST = []
NER_TAG_LIST = []

tmp1_list = []
tmp2_list = []
list_with_seq = []
list_with_ner = []

In [16]:
DATA_TEST['title,intro'].apply(element_distributor)
print('ready')

ready


In [17]:
for token,tag in zip(TOKEN_LIST, NER_TAG_LIST):
  tmp1_list.append(token)
  tmp2_list.append(tag)
  if token == '.':
    list_with_seq.append(tmp1_list)
    list_with_ner.append(tmp2_list)
    tmp1_list=[]
    tmp2_list=[]

In [18]:
test_df['ner_tags'] = list_with_ner
test_df['tokens'] = list_with_seq

In [19]:
TOKEN_LIST = []
NER_TAG_LIST = []

tmp1_list = []
tmp2_list = []
list_with_seq = []
list_with_ner = []

In [20]:
DATA_VAL['title,intro'].apply(element_distributor)
print('ready')

ready


In [21]:
for token,tag in zip(TOKEN_LIST, NER_TAG_LIST):
  tmp1_list.append(token)
  tmp2_list.append(tag)
  if token == '.':
    list_with_seq.append(tmp1_list)
    list_with_ner.append(tmp2_list)
    tmp1_list=[]
    tmp2_list=[]

In [22]:
val_df['ner_tags'] = list_with_ner
val_df['tokens'] = list_with_seq

In [23]:
test_df

,ner_tags,tokens
0,"[O, O, O, O, O, O, O, O, O, O, O, O, O]","[Тогда, замешанные, в, скандале, прокуроры, от..."
1,"[O, O, O, B-ORG, O, O, B-ORG, I-ORG, B-LOC, B-...","[Символичным, назвал, председатель, РФМ, ,, де..."
2,"[O, O, O, O, O, O, B-PER, I-PER, O, B-PER, I-P...","[На, посту, гендиректора, Yahoo, !, B-ORG, Кэр..."
3,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Считаю, невозможным, руководить, областью, с,..."
4,"[B-PER, O, B-PER, I-PER, O, I-PER, O, O, O, O,...","[Боксер, победила, Карли, Фиорину, (, Carly, F..."
...,...,...
2563,"[O, B-LOC, B-PER, I-PER, O, O, O, O, O, O, B-P...","[Президент, Чехии, Милош, Земан, во, вторник, ..."
2564,"[O, O, O, O, O, O, O, O, B-ORG, I-ORG, B-PER, ...","[``, Специальному, городу, —, специальный, реж..."
2565,"[O, B-PER, O, O, O, O, B-PER, I-PER, O, O, B-P...","[Кандидатура, Дубинина, уже, согласована, с, п..."
2566,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[Ему, придется, ``, поближе, познакомиться, с,..."


# Преобразование датасетов

Преобразуем датасеты из формата pandas dataframe в dataset

In [24]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

In [25]:
features = train_dataset.features
column_names = train_dataset.column_names
text_column_name = "tokens"
label_column_name = "ner_tags"

def get_label_list(labels):
    unique_labels = set()
    for label in labels:
        unique_labels = unique_labels | set(label)
    label_list = list(unique_labels)
    label_list.sort()
    return label_list

if isinstance(features[label_column_name].feature, ClassLabel):
    label_list = features[label_column_name].feature.names
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
else:
    label_list = get_label_list(train_dataset[label_column_name])
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
num_labels = len(label_list)

In [26]:
label_names = list(id2label.values())

In [27]:
def conver_label_to_id(text_labels):
  id_list = []
  for label in text_labels:
    for k,v in label2id.items():
      if  k == label:
        id_list.append(v)
  text_labels = id_list
  return  text_labels

In [28]:
train_df['ner_tags'] = train_df['ner_tags'].apply(conver_label_to_id)

In [29]:
test_df['ner_tags'] = test_df['ner_tags'].apply(conver_label_to_id)

In [30]:
val_df['ner_tags'] = val_df['ner_tags'].apply(conver_label_to_id)

In [31]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

# Создание модели

Используем предобученную модель bert

In [32]:
model_checkpoint = "bert-base-multilingual-cased"

In [33]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint, id2label=id2label, label2id=label2id,)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Токенизация датасетов для обучения

In [34]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for i in word_ids:
        if i == None:
          new_labels.append(-100)
        else:
          new_labels.append(labels[i])

    return new_labels

In [35]:
inputs = tokenizer(train_dataset["tokens"][0],is_split_into_words=True)
inputs.tokens()

['[CLS]',
 '"',
 'Если',
 'Мир',
 '##онов',
 'занял',
 'сто',
 '##ль',
 'о',
 '##п',
 '##по',
 '##зици',
 '##он',
 '##ную',
 'позицию',
 ',',
 'то',
 'мне',
 'представляет',
 '##ся',
 ',',
 'что',
 'для',
 'него',
 'было',
 'бы',
 'пор',
 '##яд',
 '##о',
 '##чным',
 'и',
 'правил',
 '##ьным',
 'у',
 '##йти',
 'в',
 'отставку',
 'с',
 'за',
 '##нима',
 '##емого',
 'им',
 'поста',
 ',',
 'поста',
 ',',
 'который',
 'пред',
 '##ост',
 '##ав',
 '##лен',
 'ему',
 'сегодня',
 '"',
 'Е',
 '##дино',
 '##й',
 'Р',
 '##ос',
 '##сией',
 "'",
 "'",
 'и',
 'ни',
 '##ке',
 '##м',
 'больше',
 "'",
 "'",
 ',',
 '-',
 'за',
 '##кл',
 '##ю',
 '##чает',
 'И',
 '##са',
 '##ев',
 '.',
 '[SEP]']

In [36]:
word_ids = inputs.word_ids()

In [37]:
labels = train_dataset["ner_tags"][0]

In [38]:
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[6, 6, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 4, 6, 6, 6, 6, 6, 6, 6, 6, 2, 6]
[-100, 6, 6, 2, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 2, 2, 6, -100]


In [39]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [40]:
tokenized_train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)

  0%|          | 0/8 [00:00<?, ?ba/s]

In [41]:
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [42]:
tokenized_val_dataset = val_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [43]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [44]:
batch = data_collator([tokenized_train_dataset[i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    6,    6,    2,    2,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    1,    1,    1,    4,    4,    4,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    2,    2,    2,    6, -100],
        [-100,    6,    6,    6,    1,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    2,    2,    2,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    0,    0,    0,    0,    0,
            6,    6,    6, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, 

# Подготовка метрик

In [45]:
metric = evaluate.load("seqeval")

In [46]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

# Настройка модели Token Classification

In [47]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [48]:
args = TrainingArguments(
    model_checkpoint,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# Обучение

In [49]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/huggingface_hub/repository.py:725: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  warnings.warn(
Cloning https://huggingface.co/georgeman31/bert-base-multilingual-cased into local empty directory.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7701
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2889
  Number of trainable parameters = 177268231


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.140000,0.049890,0.937586,0.952133,0.944804,0.983995
2,0.025800,0.045324,0.958900,0.969919,0.964378,0.990267
3,0.011400,0.040085,0.972172,0.976725,0.974443,0.992108


***** Running Evaluation *****
  Num examples = 2569
  Batch size = 8
Saving model checkpoint to bert-base-multilingual-cased/checkpoint-963
Configuration saved in bert-base-multilingual-cased/checkpoint-963/config.json
Model weights saved in bert-base-multilingual-cased/checkpoint-963/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-cased/checkpoint-963/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased/checkpoint-963/special_tokens_map.json
tokenizer config file saved in bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2569
  Batch size = 8
Saving model checkpoint to bert-base-multilingual-cased/checkpoint-1926
Configuration saved in bert-base-multilingual-cased/checkpoint-1926/config.json
Model weights saved in bert-base-multilingual-cased/checkpoint-1926/pytorch_model.bin
tokenizer config file saved i

TrainOutput(global_step=2889, training_loss=0.0432491112679666, metrics={'train_runtime': 606.5296, 'train_samples_per_second': 38.09, 'train_steps_per_second': 4.763, 'total_flos': 886705204684134.0, 'train_loss': 0.0432491112679666, 'epoch': 3.0})

In [50]:
trainer.push_to_hub(commit_message="Training complete")

Saving model checkpoint to bert-base-multilingual-cased
Configuration saved in bert-base-multilingual-cased/config.json
Model weights saved in bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/676M [00:00<?, ?B/s]

Upload file runs/Jan14_13-37-15_d2ada6bdd6c6/events.out.tfevents.1673703448.d2ada6bdd6c6.187.0:  50%|#####    …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/georgeman31/bert-base-multilingual-cased
   efbcb35..4af00c2  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/georgeman31/bert-base-multilingual-cased
   efbcb35..4af00c2  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.9721716325489911}, {'name': 'Recall', 'type': 'recall', 'value': 0.9767254629290786}, {'name': 'F1', 'type': 'f1', 'value': 0.9744432274552758}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9921081706739622}]}
To https://huggingface.co/georgeman31/bert-base-multilingual-cased
   4af00c2..f8f0a3b  main -> main

   4af00c2..f8f0a3b  main -> main



'https://huggingface.co/georgeman31/bert-base-multilingual-cased/commit/4af00c2d13640b06f463d5fbd46563aa8a25d2f4'

In [51]:
train_dataloader = DataLoader(
    tokenized_train_dataset,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_val_dataset, collate_fn=data_collator, batch_size=8
)

In [52]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

loading configuration file bert-base-multilingual-cased/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-LOC",
    "1": "B-ORG",
    "2": "B-PER",
    "3": "I-LOC",
    "4": "I-ORG",
    "5": "I-PER",
    "6": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 0,
    "B-ORG": 1,
    "B-PER": 2,
    "I-LOC": 3,
    "I-ORG": 4,
    "I-PER": 5,
    "O": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_ty

In [53]:
optimizer = AdamW(model.parameters(), lr=2e-5) # настройка оптимайзера

In [55]:
accelerator = Accelerator() # акселератор модели
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [56]:
num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

Сохранение в huggingface

In [57]:
model_name = "bert-finetuned-ner-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

'georgeman31/bert-finetuned-ner-accelerate'

In [58]:
output_dir = "bert-finetuned-ner-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.8/dist-packages/huggingface_hub/repository.py:725: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  warnings.warn(
Cloning https://huggingface.co/georgeman31/bert-finetuned-ner-accelerate into local empty directory.


Постобработка модели

In [59]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [60]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # оценка модели
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Сохранение и загрузка
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/2889 [00:00<?, ?it/s]

Configuration saved in bert-finetuned-ner-accelerate/config.json


epoch 0: {'precision': 0.9352997145575642, 'recall': 0.937908256880734, 'f1': 0.9366021694517738, 'accuracy': 0.9845530593991687}


Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner-accelerate/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner-accelerate/special_tokens_map.json
Configuration saved in bert-finetuned-ner-accelerate/config.json


epoch 1: {'precision': 0.9740174193076191, 'recall': 0.9646973541138093, 'f1': 0.9693349843397189, 'accuracy': 0.9907928657862892}


Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner-accelerate/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner-accelerate/special_tokens_map.json
Several commits (2) will be pushed upstream.
Configuration saved in bert-finetuned-ner-accelerate/config.json


epoch 2: {'precision': 0.9768718436653736, 'recall': 0.9709733740724574, 'f1': 0.9739136779889818, 'accuracy': 0.9919503340874415}


Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner-accelerate/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner-accelerate/special_tokens_map.json
Several commits (3) will be pushed upstream.


In [61]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

Configuration saved in bert-finetuned-ner-accelerate/config.json
Model weights saved in bert-finetuned-ner-accelerate/pytorch_model.bin


In [62]:
model_checkpoint = "/content/bert-base-multilingual-cased"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint
)

loading configuration file /content/bert-base-multilingual-cased/config.json
Model config BertConfig {
  "_name_or_path": "/content/bert-base-multilingual-cased",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-LOC",
    "1": "B-ORG",
    "2": "B-PER",
    "3": "I-LOC",
    "4": "I-ORG",
    "5": "I-PER",
    "6": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 0,
    "B-ORG": 1,
    "B-PER": 2,
    "I-LOC": 3,
    "I-ORG": 4,
    "I-PER": 5,
    "O": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head":

In [63]:
test_dataset

Dataset({
    features: ['ner_tags', 'tokens'],
    num_rows: 2568
})

# Классификация

In [72]:
example = test_dataset['tokens'][2]

In [74]:
' '.join(example)

'На посту гендиректора Yahoo ! B-ORG Кэрол Бартц сменила Джерри Янга - основателя компании .'

In [75]:
token_classifier('На посту гендиректора Yahoo ! B-ORG Кэрол Бартц сменила Джерри Янга - основателя компании .')

[{'entity': 'B-PER',
  'score': 0.9997813,
  'index': 13,
  'word': 'К',
  'start': 36,
  'end': 37},
 {'entity': 'B-PER',
  'score': 0.9997743,
  'index': 14,
  'word': '##эр',
  'start': 37,
  'end': 39},
 {'entity': 'B-PER',
  'score': 0.99978787,
  'index': 15,
  'word': '##ол',
  'start': 39,
  'end': 41},
 {'entity': 'I-PER',
  'score': 0.9997979,
  'index': 16,
  'word': 'Ба',
  'start': 42,
  'end': 44},
 {'entity': 'I-PER',
  'score': 0.9997999,
  'index': 17,
  'word': '##рт',
  'start': 44,
  'end': 46},
 {'entity': 'I-PER',
  'score': 0.9997937,
  'index': 18,
  'word': '##ц',
  'start': 46,
  'end': 47},
 {'entity': 'B-PER',
  'score': 0.99980015,
  'index': 22,
  'word': 'Дж',
  'start': 56,
  'end': 58},
 {'entity': 'B-PER',
  'score': 0.9997913,
  'index': 23,
  'word': '##ер',
  'start': 58,
  'end': 60},
 {'entity': 'B-PER',
  'score': 0.99980015,
  'index': 24,
  'word': '##ри',
  'start': 60,
  'end': 62},
 {'entity': 'I-PER',
  'score': 0.9997609,
  'index': 25,
  